# Unsupervised PDE solving techniques
First I'll set some very basic notation. I will assume that we are interested in PDEs of the form $L(x,u,c) = 0$ where:
- $L$ represents the PDE/model
- $x$ is the spatial coordinate, say $x\in\Omega$
- $u$ is the solution, say $u\colon \Omega\to\mathbb R$
- $c$ is an inhomogeneous model parameter, say $c\colon \Omega\to\mathbb R^n$

For example, we could have $L(x,u,c) = \Delta u(x) - c(x)$ or $L(x,u,c) = \nabla\cdot(c_1(x)\nabla u) - c_2(x)$. In the Navier-Stokes equation $c$ would represent pressure or density etc.

There are many contrasting interpretations of ''unsupervised'' in the PDE setting:
1. The PDE is given, no example solutions are given
   1. Compute interior solution from given boundary data ([PINN 1](https://github.com/Computational-physics-with-learning/PINNs), [PINN 2](https://arxiv.org/pdf/2110.13297.pdf), DIP, [physics loss 1](https://arxiv.org/pdf/2107.03673.pdf), [physics loss 2](https://arxiv.org/pdf/2110.01601v2.pdf))
   2. Compute solution from interior observations (e.g. for Navier-Stokes, observations of pressure/density) ([PINN](https://github.com/Computational-physics-with-learning/PINNs), DIP, [physics loss 1](https://arxiv.org/pdf/2107.03673.pdf), [physics loss 2](https://arxiv.org/pdf/2110.01601v2.pdf))
   3. Solution can be on a fixed grid (DIP) or mesh free 
   4. <span style="background-color:#6600CC">[Un-paired solutions and coefficient maps](#1.4-unsupervised-with-unpaired-data)</span>
2. Example solutions are given, no PDE is given
   1. Extract a white box constraint for solutions (e.g. an explicit polynomial PDE, [SINDy](https://github.com/Computational-physics-with-learning/pde-model-discovery/blob/main/pysindy.ipynb), [Symbolic Regression](https://github.com/Computational-physics-with-learning/pde-model-discovery/blob/main/SymbolicRegression.ipynb), [PDE-net](https://arxiv.org/pdf/1710.09668.pdf))
   2. <span style="background-color:#6600CC"> [Extract a grey box constraint for solutions](#2.2-Learning-solution-maps-without-the-PDE)</span> (e.g. neural network with 'PDE-like architecture' as function of solution and its derivatives, [Neural ODE](https://arxiv.org/pdf/1806.07366.pdf)) 
   3. Extract a black box constraint for solutions (e.g. generic neural network, maybe for time-stepping scheme, in which case it is basically supervised learning)
   4. Extract a model with particular structure (e.g. Hamiltonian/Lagrangian, [Hamiltonian NN](https://arxiv.org/pdf/1906.01563.pdf))
3. Transfer-learning
   1. Trained for one input data, adapted to new data ([PINN 1](https://aia.springeropen.com/articles/10.1186/s42774-021-00094-7), [PINN 2](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9826396), [DeepONet](https://www.nature.com/articles/s42256-022-00569-2))
   2. Supervised on synthetic example, adapted to real (microscope) data 
4. Image quality
   1. Super-resolution
   2. Denoising

Quite a nice review on `Physics-Informed Machine learning` (PIML) is [here](https://arxiv.org/pdf/2211.08064.pdf). It is surprisingly common to think that PIML = PINN...
TODO: DIP

# 2.B Learning solution maps without the PDE
This section follows the paper [Data-driven discovery of Green’s functions with human-understandable deep learning](https://arxiv.org/pdf/2105.00266.pdf) by Boulle, Earls, and Townsend. There are several works which only use the PDE loss under the tags `deep` and `Green's function`, for example [here](https://arxiv.org/pdf/2107.03673.pdf).

The problem considered can be written as:
$$\text{Find }\quad u\quad \text{ such that }\quad L(x,u) = c(x)\quad \text{ for } x\in \Omega,\qquad u(x)= 0 \quad\text{ for } x\in\partial\Omega.$$
If $L$ is linear, then the solution is well known. There is a Green's function $G\colon \Omega^2\to\mathbb{R}$ such that
$$ \forall x\in \Omega,\qquad L(x,u) = c \implies u(x) = \int_\Omega G(x,y) c(y) dy + u_0(x) .$$
In practice, you don't need to know $L$, and it doesn't need to be linear. The Greens function just motivates an unsupervised formulation. In particular, we parametrise $u_0 \gets \varphi_\theta$ and $G \gets \psi_\eta$ as neural networks with parameters $\theta$ and $\eta$. Then, the loss is
$$ \operatorname{loss}(\theta,\psi) =  \sum_{(u,c)} \|u\|_2^{-2}\left\|u - \varphi_\theta - \int_\Omega \psi_\eta(\cdot,y) c(y) dy\right\|^2. $$
This is unsupervised because the (unknown) operator $L$ does not appear.

### Result 1: $G$ and $u_0$ are learnable
Consider the 1D Helmholtz equation, $L(x,u) = u''(x) + 15^2 u(x)$, where the known Green's function is known analytically.
<img style="background-color:white;max-width:750px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/UnderstandableGreen/FigS3.jpg">  
Figure S3: Different approximations for Helmholtz equation using different activation functions    
.  
<img style="background-color:white;max-width:750px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/UnderstandableGreen/FigS7.jpg">  
Figure S7: Different approximations for Laplace (top) and Advection (bottom) equations

The rational NN uses activation functions of the form $\sigma(x) = \operatorname{cubic}(x)/\operatorname{quadratic}(x)$, apparently more efficient, more accurate, and poles are informative.



### Result 2: the learned $G$ is like a truncated SVD
We can compute singular values/vectors of the exact/learned Green's functions and compare them, they are somehow related to the SVD of the original $L$.

.  
<img style="background-color:white;max-width:750px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/UnderstandableGreen/FigS8.jpg">  
Figure S8: Exact (top) and learned (bottom) singular values (left) and singular vectors (right) for Laplace equation  
.  
<img style="background-color:white;max-width:750px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/UnderstandableGreen/FigS9.jpg">  
Figure S9: SVD for exact (top) and learned (bottom) advection equation. Left/right singular vectors are shown left/right and singular values in the middle

### Other details:
- Figure S4, performance relative to number of training pairs, size of training data, amount of noise
- Table S1, comparing loss discretisation methods with different point distributions and quadrature rules
- Figure S11, location of singularities in rational network, potentially informative of singularities of true system
- Figure S13, doesn't work so well in 2D
- Section 9, solving time-dependant systems with time-stepping scheme

### Summary:
- You can use the Green's function formulation to learn an inverse operator from data without knowing $L$ explicitly
- Performance seems robust to hyperparameters, but limited to around $10^{-3}-10^{-4}$ error
- Seems to have a nice SVD interpretation (grey-box)

# 1.D Unsupervised with unpaired data
This section follows the paper [DeepGreen: Deep Learning of Green’s Functions for Nonlinear Boundary Value Problems](https://arxiv.org/pdf/2101.07206.pdf) by Gin, Shea, Brunton, and Kutz.
The usecase I'm envisaging is:
- The operator $L$ is known but expensive to evaluate (non-linear $L(u) = c$)
- There are a few paired simulations $(u_n,c_n)$, more un-paired samples $u^k$, $c^k$
- We want to learn approximations to the forward and inverse mappings, $u\leftrightarrow c$

The proposed approach is:
- learn autoencoders $\varphi_\theta$ and $\psi_\eta$ such that $\varphi_\theta^{-1}(\varphi_\theta(u)) \approx u$, $\psi_\eta^{-1}(\psi_\eta(c)) \approx c$ for 
- learn symmetric linear map $A$ so that $L(u) \approx \psi_\eta^{-1}(A\varphi_\theta(u))$, so the "Green's" function is $G(c) \approx \varphi_\theta^{-1}(A^{-1}\psi_\eta(c))$.

The first part is unsupervised, we can use $u=u_n$ or $u=u^k$ etc., and the second is supervised requiring pairs $(u_n,c_n)$.
<img style="background-color:white;max-width:1000px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/DeepGreen/Fig1.png">  
Figure 1: The original physical problem may be very complicated. But, with a couple of non-linear changes of basis, it may be very simple.

### Observation 1: Training behaviour
A typical training curve looks like this:  
<img style="background-color:white;max-width:750px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/DeepGreen/Fig3.png">  
Figure 3: Typical convergence behaviour.

- In the first 75 iterations only the autoencoders are trained, then the linear pairing too
- Accuracy only reaches $\sim10^{-4}$, not great. Apparently hyperparameters were not heavily tuned.
- Generalisation error is good

### Observation 2: Diagonalised physics
The learned matrix $A$ is typically near-diagonal. Apparently this doesn't depend strongly on the initialisation. 

<img style="background-color:white;max-width:750px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/DeepGreen/Fig5.png">  

Figure 5: Visualisation of $A^{-1}$ and $A$ respectively.

### Example results
Results look pretty reasonable. They work best with simple sinusoidal data, worst with more singular. The training data was a mix of $c$ either a sinusoid or Gaussian.  
<img style="background-color:white;max-width:1000px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/DeepGreen/Fig6.png">  
Figure 6: Their $F$ is our $c$.

Out-of-sample tests degrade a little, here it is tested on cubic $c$.  
<img style="background-color:white;max-width:1000px" src="https://github.com/Computational-physics-with-learning/unsupervised-learning/raw/main/DeepGreen/Fig7.png">  
Figure 7: Out of training test. Again, their $F$ is our $c$.

The pictures look pretty similar in 2D 

### Summary:
- For a given physical system we can ''diagonalise'' the model by using autoencoders
- Autoencoders are a great way of incorporating unpaired data into the learning
- Again, very hard to get beyond $10^{-3}-10^{-4}$ accuracy